# Wires game(s)

This noteboook will generate a contitional wires game with the text instructions and arduino code

In [2]:
from copy import deepcopy
import random
from typing import Any, Dict, List

In [3]:
states_dict = {
    'open': {
        'sensor_val': 0,
        'mapped_val': 0,
        'mapped_opposite': 0,
        'manipulation': 'pull out the {{wire}} wire'
    },
    'low': {
        'sensor_val': 100,
        'mapped_val': 1,
        'mapped_opposite': 2,
        'manipulation': 'plug the {{wire}} wire into the bottom rail'
    },
    'high': {
        'sensor_val': 500,
        'mapped_val': 2,
        'mapped_opposite': 1,
        'manipulation': 'plug the {{wire}} wire into the top rail'
    }
}

states = list(states_dict.keys())

In [4]:
wires = [
    'red',
    'yellow',
    'orange',
    'white',
    'blue',
    'green',
    # 'black'
]

wires_dict = {wires[i]: f'A{i}' for i in range(len(wires))}
wires_dict

{'red': 'A0',
 'yellow': 'A1',
 'orange': 'A2',
 'white': 'A3',
 'blue': 'A4',
 'green': 'A5'}

In [5]:
combis_list = []

for s in states:
    for w in wires:
        combis_list.append([s, w])
        
combis_list

[['open', 'red'],
 ['open', 'yellow'],
 ['open', 'orange'],
 ['open', 'white'],
 ['open', 'blue'],
 ['open', 'green'],
 ['low', 'red'],
 ['low', 'yellow'],
 ['low', 'orange'],
 ['low', 'white'],
 ['low', 'blue'],
 ['low', 'green'],
 ['high', 'red'],
 ['high', 'yellow'],
 ['high', 'orange'],
 ['high', 'white'],
 ['high', 'blue'],
 ['high', 'green']]

In [6]:
lights = [
    'top',
    'middle',
    'bottom'
]

light_states = {
    'on': 'HIGH',
    'off': 'LOW'
}

lights_dict = {lights[i]: f'{i + 2}' for i in range(len(lights))}
lights_dict

{'top': '2', 'middle': '3', 'bottom': '4'}

In [7]:
code_template = """

/*
  Auto-generated wiring program template for wires game.

  Reads against 3 states:
   - High rail connected
   - Low rail connected
   - Disconnected

  High rail should be connected to digital pin 5.
  Low rail should be connected to digital pin 6.
  All input wires should have 100K pull-down to GND, then conncted to inputs specified below. 
*/


const int win_sig_out = 10;
const int lose_sig_out = 11;
const int lose_sig_in = 12;

const int high_rail = 7;
const int low_rail = 8;

int seed_pin = A6;
int ee_seed = 0;

{{wire_pin_declarations}}

bool wire_change = false;
int progress = 0;
int mistake_count = 0;
int lose_count = 3;


int get_curr_pin_state(int pin) {
  // Gets the current mapped state of a given pin
  // Maps the input value against uneven thresholds for 3-level mapping
  // open = {{open_map}}, low = {{low_map}}, high = {{high_map}}
  int val_in = {{open_map}};
  digitalWrite(high_rail, HIGH);
  if (digitalRead(pin) == HIGH) {
    val_in = {{high_map}};
  }
  digitalWrite(high_rail, LOW);
  digitalWrite(low_rail, HIGH);
  if (digitalRead(pin) == HIGH) {
    val_in = {{low_map}};
  }
  digitalWrite(low_rail, LOW);
  return val_in;
}


void safe() {
  // Actions upon game won
  digitalWrite(win_sig_out, HIGH);
}


void lose () {
  // Actions upon game lose
  digitalWrite(lose_sig_out, HIGH);
  digitalWrite(top, LOW);
  digitalWrite(middle, LOW);
  digitalWrite(bottom, LOW);
  digitalWrite(high_rail, LOW);
  digitalWrite(low_rail, LOW);
  
}


{{game_functions}}


void setup() {
  // setup random seed
  ee_seed = analogRead(seed_pin);
  randomSeed(ee_seed);
{{setup_code}}
}


void loop() {
  // Game loop
{{loop_code}}
}

"""

jinjas_ref = {
    'open_map': states_dict['open']['mapped_val'],
    'low_map': states_dict['low']['mapped_val'],
    'high_map': states_dict['high']['mapped_val']
    
    
}

for jinja, value in jinjas_ref.items():
    code_template = code_template.replace(f'{{{{{jinja}}}}}', str(value))

# print(code_template)

# Game 1: Sequence game

In [9]:
sequence_game_code_functions = """
void check_progress() {
  // Checks the progress and increments if successful
  bool correct = false;
  switch (progress) {
{{case_statements}}
  }
  
  if (correct == true) {
      // set curr_pin_states
      for (int i=0; i<{{num_wires}}; i++) {
        int curr_pin_state = get_curr_pin_state(wires[i]);
        int prev_pin_state = curr_pin_states[i];
      }
      progress ++;
  }
  Serial.print("Progress: ");
  Serial.println(progress);
}
"""


sequence_game_setup = """
  Serial.begin(9600);
  // set the current wire conditions
  for (int i=0; i<{{num_wires}}; i++) {
    int curr_pin_state = get_curr_pin_state(wires[i]);
    curr_pin_states[i] = curr_pin_state;
  }
  // setup the win / lose signal wires
  pinMode(win_sig_out, OUTPUT);
  digitalWrite(win_sig_out, LOW);
  pinMode(lose_sig_out, OUTPUT);
  digitalWrite(lose_sig_out, LOW);
  pinMode(lose_sig_in, INPUT);
  // setup the rail pins;
  pinMode(high_rail, OUTPUT);
  digitalWrite(high_rail, LOW);
  pinMode(low_rail, OUTPUT);
  digitalWrite(low_rail, LOW);
  // set the sequence indicator LEDs
{{sequence indicator setup}}
  // set the wire pins to be digital
{{wire pin setup}}

"""


sequence_game_loop = """
  // check if lost elsewhere
  if (lose_sig_in == HIGH) {
    mistake_count == lose_count;
  }
  // check for wire change
  wire_change = false;
  for (int i=0; i<{{num_wires}}; i++) {
    allowed_mistakes[i] = true;
  }
  if (mistake_count < lose_count) {
    for (int i=0; i<{{num_wires}}; i++) {
      int curr_pin_state = get_curr_pin_state(wires[i]);
      int prev_pin_state = curr_pin_states[i];
      if (curr_pin_state != prev_pin_state) {
        wire_change = true;
        curr_pin_states[i] = curr_pin_state;
        allowed_mistakes[i] = false;
        Serial.print("A");
        Serial.print(i);
        Serial.print(": ");
        Serial.println(curr_pin_state);
        delay(100);
      }
    }
  }  if (wire_change == true && mistake_count < lose_count) {
    check_progress();
  }

  if (progress == 5) {
    safe();
  }
  else {
    if (mistake_count == lose_count) {
      lose();
    }
  }
"""

# Adjust spacing to better suit sketch layout
sequence_game_code_functions = sequence_game_code_functions[1:]
sequence_game_code_functions = sequence_game_code_functions[:-1]
sequence_game_setup = sequence_game_setup[1:]
sequence_game_setup = sequence_game_setup[:-1]
sequence_game_loop = sequence_game_loop[1:]
sequence_game_loop = sequence_game_loop[:-1]

In [10]:

class sequence_object:
    """
    Stores an instance of a sub-game (sequence isntructions and corresponding hardware attributes) 
    """
    def __init__(self,
                 code_template: str,
                 num_instructions: int = 5) -> None:
        """
        :param num_instructions: the length of the sequence to be carried out in instruction steps
        """
        self.num_instructions: int = num_instructions
        self.instructions_list = []
        self.instruction_states = []
        
        self.start_lights = {}
        self.start_lights = {}
        self.used_start_light_states = []
        
        self._required_wire_states = {}
        
        self.code_template: str = code_template
        self.sequence_game_setup = sequence_game_setup
        self.game_code_functions: str = sequence_game_code_functions
    
    
    def generate_start_lights(self) -> None:
        """
        Function produces the indicating start light configuration
        """
        unused_start_lights = False
        while not unused_start_lights:
            self.start_lights = [random.choice(list(light_states.keys())) for light in lights]
            if self.start_lights not in self.used_start_light_states:
                self.used_start_light_states.append(self.start_lights)
                unused_start_lights = True
    
    
    def generate_start_light_text(self) -> str:
        """
        Function produces the indicating start light configuration
        """
        if not self.start_lights:
            self.generate_start_lights()
        return "\n".join(self.start_lights)
    
    
    def generate_step(self) -> None:
        """
        Genererates an indivudal step instructions and states
        """
        num_step_wires = random.randint(1, 2)
        step_wires = {}
        wire_instructions = []
        for i in range(num_step_wires):
            step_wires[random.choice([wire for wire in wires if wire not in step_wires])] = {}
            
        for wire in step_wires:
            
            wire_state = random.choice([state for state in states_dict
                                        if state != self._required_wire_states.get(wire, None)])
            step_wires[wire] = wire_state
            self._required_wire_states[wire] = wire_state
            
            wire_instruction = states_dict[wire_state]['manipulation'].replace('{{wire}}', wire)
            wire_instructions.append(wire_instruction)
        
        step_instruction = " and ".join(wire_instructions)
        step_instruction_formatted = f'P{step_instruction[1:]}.'
        
        self.instructions_list.append(step_instruction_formatted)
        self.instruction_states.append(deepcopy(self._required_wire_states))
            
        # print(step_wires)
        # print(wire_instructions)
        # print('-------')
        # print(self._required_wire_states)
        # print(self.instructions_list)
        # print('')

        
    def generate_sequence(self) -> None:
        """
        Recursively generates the required number of instructions
        """
        for i in range(self.num_instructions):
            self.generate_step()
    
    
     # --------------------------------
        # Code genetarors
     # --------------------------------
    
    def generate_pin_declarations(self) -> None:
        """
        Auto-generates pin declarations for arduino code
        """
        # declare individual wire pins
        pin_declarations = []
        for wire, pin in wires_dict.items():
            pin_declarations.append(f"const int {wire} = {pin};")
        
        # declare pin array
        pin_array_str = ', '.join([f'{wire}' for wire in wires])
        pin_declarations.append(f"\nconst int wires[{len(wires)}] = {{{pin_array_str}}};")
        
        # declare pin states
        pin_declarations.append(f"\nint curr_pin_states[{len(wires)}] = {{{', '.join(['0'] * len(wires))}}};")
        pin_declarations.append(f"int allowed_mistakes[{len(wires)}] = {{{', '.join(['true'] * len(wires))}}};")
        
        pin_declarations.append('\n//Start light pin declarations')
        # declare start light pins
        start_lights = [f"const int {light} = {lights_dict[light]};" for light in lights]
        pin_declarations.extend(start_lights)
        
        pin_dec_str = "\n".join(pin_declarations)
        self.code_template = self.code_template.replace('{{wire_pin_declarations}}', pin_dec_str)
        
    
    def generate_start_light_setup(self,
                                   num: int = 0)->None:
        """
        Code generation function for producing start light setup
        :param num: which start light sequence to use
        """
        light_list = self.used_start_light_states[num]
        setup_lines = []
        for i in range(len(lights)):
            setup_lines.append(f'  pinMode({lights[i]}, OUTPUT);')
            setup_lines.append(f'  digitalWrite({lights[i]}, {light_states[light_list[i]]});')
        
        self.sequence_game_setup = self.sequence_game_setup.replace('{{sequence indicator setup}}',
                                                                    '\n'.join(setup_lines))
        
        
    
    def generate_correct_conditions_logic(self) -> None:
        """
        Code generation function for producing game condition logic
        """
        case_template = """    case {{case_num}}:
      if ({{correct_conditions}}) {
        correct = true;
      }
      else {
        if ({{incorrect_conditions}}) {
          mistake_count++;
          Serial.println("Oops!");
        }
      }
      break;"""
        
        cases = []
        for i in range(self.num_instructions):
            # generate correct conditions check for each instruction set
            correct_case_conditions = []
            incorrect_case_conditions = []
            for wire, state in self.instruction_states[i].items():
                correct_case_conditions.append(f'curr_pin_states[{wires.index(wire)}] == {states_dict[state]["mapped_val"]}')
                if state != 'open':
                    incorrect_case_conditions.append(f'(curr_pin_states[{wires.index(wire)}] == {states_dict[state]["mapped_opposite"]}' \
                                                     f' && allowed_mistakes[{wires.index(wire)}] == false)')
                    
            correct_conditions_str = ' && '.join(correct_case_conditions)
            incorrect_conditions_str = ' || '.join(incorrect_case_conditions)            
            case_i = case_template.replace('{{case_num}}', str(i))
            case_i = case_i.replace('{{correct_conditions}}', correct_conditions_str)
            case_i = case_i.replace('{{incorrect_conditions}}', incorrect_conditions_str)
            cases.append(case_i)
            
        self.game_code_functions = self.game_code_functions.replace('{{case_statements}}', '\n'.join(cases))
            
            
        # print('\n'.join(case_i))
            # ToDo: Generate allowed open states here <<<<<<<<<<<<<<<<<<<<<<<<
                # include current instruction's colours as open allowed
                # needs to check if any allowed-opens are not open
            
                
    
    
    def generate_game_code(self) -> None:
        """
        Code generation function for producing the complete arduino code
        """
        self.generate_pin_declarations()
        self.generate_start_light_setup()
        self.code_template = self.code_template.replace('{{game_functions}}', self.game_code_functions)        
        
        wire_setup_lines = [f"  pinMode({wire}, INPUT);" for wire in wires]
        self.sequence_game_setup = self.sequence_game_setup.replace('{{wire pin setup}}',
                                                                    '\n'.join(wire_setup_lines))
        self.code_template = self.code_template.replace('{{setup_code}}', self.sequence_game_setup)

        self.code_template = self.code_template.replace('{{loop_code}}', sequence_game_loop)
        
        # update any for-loop lengths dependent upon number of wires
        self.code_template = self.code_template.replace('{{num_wires}}', str(len(wires)))
        print(self.code_template)
        


In [11]:
test_so = sequence_object(code_template)

In [12]:
print(test_so.generate_start_light_text())

on
on
off


In [13]:
test_so.generate_sequence()

In [14]:
test_so.instructions_list

['Plug the yellow wire into the top rail and plug the red wire into the bottom rail.',
 'Plug the white wire into the bottom rail.',
 'Plug the orange wire into the top rail.',
 'Plug the red wire into the top rail and plug the green wire into the bottom rail.',
 'Plug the red wire into the bottom rail and pull out the white wire.']

In [15]:
test_so.instruction_states

[{'yellow': 'high', 'red': 'low'},
 {'yellow': 'high', 'red': 'low', 'white': 'low'},
 {'yellow': 'high', 'red': 'low', 'white': 'low', 'orange': 'high'},
 {'yellow': 'high',
  'red': 'high',
  'white': 'low',
  'orange': 'high',
  'green': 'low'},
 {'yellow': 'high',
  'red': 'low',
  'white': 'open',
  'orange': 'high',
  'green': 'low'}]

In [16]:
wires_dict

{'red': 'A0',
 'yellow': 'A1',
 'orange': 'A2',
 'white': 'A3',
 'blue': 'A4',
 'green': 'A5'}

In [17]:
test_so.generate_correct_conditions_logic()

---

In [19]:
test_so.generate_game_code()



/*
  Auto-generated wiring program template for wires game.

  Reads against 3 states:
   - High rail connected
   - Low rail connected
   - Disconnected

  High rail should be connected to digital pin 5.
  Low rail should be connected to digital pin 6.
  All input wires should have 100K pull-down to GND, then conncted to inputs specified below. 
*/


const int win_sig_out = 10;
const int lose_sig_out = 11;
const int lose_sig_in = 12;

const int high_rail = 7;
const int low_rail = 8;

int seed_pin = A6;
int ee_seed = 0;

const int red = A0;
const int yellow = A1;
const int orange = A2;
const int white = A3;
const int blue = A4;
const int green = A5;

const int wires[6] = {red, yellow, orange, white, blue, green};

int curr_pin_states[6] = {0, 0, 0, 0, 0, 0};
int allowed_mistakes[6] = {true, true, true, true, true, true};

//Start light pin declarations
const int top = 2;
const int middle = 3;
const int bottom = 4;

bool wire_change = false;
int progress = 0;
int mistake_count = 0;
in

ToDo:
 - allowed_opens doesn't work - needs to check if high / low is wrong for any pin (accepting open might be ok)
   - check the **changed pins** for a non-open incorrect state
 - Multiple game options with handling
 - Do POC for one working game first
 - See if arduino can handle referenced game conditions manually before building code generator